# Exploratory Data Analysis of Heavy Truck J1939 data

Ce notebook va servir de base dans l'analyse des données du `Heavy Truck`. Ces données reprennent une payload que nous pouvons retrouver avec le protocole **bus CAN J1939**.

**Payload d'un message sous protocole J1939:**

![payload j1939](./images/j1939-payload.png)

:warning: Attention à bien écouter Vendredi 26 la présentation de `Simon Bellemare` sur les données **bus CAN J1939** !

## Imports

In [1]:
import csv
from pathlib import Path

import dask.dataframe as dd
import pandas as pd
import plotly.express as px
from tqdm.notebook import tqdm

## Chargement des données

Il est important de noter que l'arborescence des données est la suivante:

```txt
|--- heavy_truck_data
    |--- part_1
        |--- part_1
            |--- csv
    |--- part_2
        |--- part_2
            |--- csv
    |--- part_3
        |--- part_3
            |--- csv
    |--- part_4
        |--- part_4
            |--- csv
```

In [ ]:
heavy_truck_data_path = Path("../data/heavy_truck_data")

ddf = dd.read_csv("../data/heavy_truck_data/**/*.csv", sep=";", on_bad_lines="skip")

## Découverte des données

### Les colonnes

Nous pouvons commencer par regarder quelles colonnes composent nos données. Cela nous donnera peut être une idée des parties du message qui sont conservés.

In [ ]:
ddf.columns

Il y a donc 3 parties claires que nous possédons dans les données:

- `id`: l'adresse de l'émetteur, du récepteur, de la priorité et/ou autre (voir la capture d'écran: PGN par exemple).
- `dlc`: le fameux Data Length Content qui indique la taille du payload.
- `data`: ce doit être la payload en elle même.

Il serait intéressant de confirmer nos intuitions par rapport au graphique du payload montré en début de notebook.

In [ ]:
ddf.dtypes

On a que des valeurs décimales, nous allons soit devoir les convertir en binaire soit les garders en décimales, excepté la colonne `timestamp`.

### Les valeurs d'ID

In [ ]:
ddf["id"].head(5)

Ce sont des valeurs réelles, sous 256 valeurs ? Vérifions.

In [ ]:
# Commenté sinon bloque l'exécution globale
# ddf["id"].min().compute(), ddf["id"].max().compute()

## Vérification des CSVs

In [ ]:
# every_df_dtypes = {}
# every_columns = []
# # Itère sur tous les CSVs dans le répertoire racine des données
# for p in (pbar := tqdm(heavy_truck_data_path.glob("**/*.csv"))):
#     pbar.set_description(p.name)
#     df = pd.read_csv(p, sep=";")
    
#     # Les colonnes
#     every_columns.extend(df.columns.tolist())
    
#     # Les types des données
#     for k, v in df.dtypes.to_dict().items():
#         if k in every_df_dtypes.keys():
#             every_df_dtypes[k].append(v)
#         else:
#             every_df_dtypes[k] = [v]

Un CSV exemple qui pose problème: [`../data/heavy_truck_data/part_1/part_1/20201123115624139152.csv`](../data/heavy_truck_data/part_1/part_1/20201123115624139152.csv).

Le problème étant que nous nous attendons à avoir 4 valeurs par ligne dans le tableau, sachant qu'il est supposé y avoir 4 colonnes (`timestamp`, `id`, `dlc`, `data`). Or, dans la plupart des lignes d'enregistrement il y a 11 valeurs, ce qui n'est pas normal.

<span style="color:red">Voir si c'est possible:</span> ajouter un traitement à l'ouverture pour gérer ce cas.

Je pense avoir compris ...

In [ ]:
working_p = next(iter(heavy_truck_data_path.glob("**/*.csv")))
pd.read_csv(working_p, sep=";")

# Comprendre le format

Le format de données n'est pas CSV (ou pandas) friendly ...

Il essaie de fitter un certain format or, ça ne marche pas avec les données actuelles car selon la valeur du `DLC` nous allons avoir plus ou moins de colonnes.

Ce qui serait pertinent, ce serait de pouvoir ouvrir les données et appliquer une fonction ligne par ligne pour pouvoir construire correctement les données.

**Voici un extrait des données ouvert avec Excel:**

![extrait données](./images/extrait-donnees-heavy-truck.png)

<span style="color:orange">Un point important est que grâce à cette capture d'écran, nous pouvons facilement observer les types de données contenues dans les colonnes, à savoir:</span>

1. Le `timestamp` du message
2. L'`id` sous format hexadecimal
3. Le `dlc` représentant la taille de la payload en octets
4. Le restant des colonnes sont pour la payload, chaque colonne supplémentaire est un octet

<span style="color:red">*Remarque:*</span> Un octet est encodé sous 8 bits, donc `2^8` valeurs, ce qui équivaut à une valeur entière dans l'intervalle [0; 255]

# Utilisons la librairie `csv` et non pas `pandas`

In [ ]:
correct_pandas_format_data = []
with open(heavy_truck_data_path.joinpath("part_1", "part_1", "20201123075441304067.csv"), mode="r") as f:
    csv_reader = csv.reader(f, delimiter=";")

    for row_id, row in enumerate(csv_reader):
        if row_id == 0: # Ne nous ennuyons pas avec le header, nous le connaissons
            continue
        
        correct_pandas_format_data.append(
            {
                "timestamp": row[0],
                "id": row[1],
                "dlc": row[2],
                "data": row[3:],
            }
        )

df_example = pd.DataFrame(correct_pandas_format_data)

# Convertir la colonne timestamp en datatime
df_example["timestamp"] = pd.to_datetime(df_example["timestamp"])
df_example

Ce format est beaucoup plus simple à manipuler pour un `DataFrame`. Nous pouvons transformer à notre guise les lignes contenues dans la colonne `data`.

Dans un premier temps, on peut convertir cette ligne de liste d'octets en base 10 sous une unique chaine binaire que nous pouvons ensuite convertir en une seule valeur réelle, pour former une valeur d'un signal.

En voici un exemple que nous pouvons ensuite appliquer pour créer nos nouvelles colonnes:

In [ ]:
payload = df_example["data"].iloc[0]
int("".join(["{:08b}" for _ in range(len(payload))]).format(*[int(val) for val in payload]), 2)

Colonne binaire:

In [ ]:
df_example["binary_data"] = df_example["data"].apply(lambda x: "".join(["{:08b}" for _ in range(len(x))]).format(*[int(val) for val in x]))

Colonne entière:

In [ ]:
df_example["int_data"] = df_example["binary_data"].apply(lambda x: int(x, 2))

Visualisons:

In [ ]:
df_example